<a href="https://colab.research.google.com/github/sahokk/AI-app/blob/main/code_multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importとか

In [6]:
from glob import glob
import numpy as np
import pandas as pd
import os
import cv2
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import tensorflow as tf
from keras import layers, models
tf.keras.backend.clear_session()

In [8]:
!pip install -qqq pyworld

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 KB 17.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
import pyworld as pw # 音声処理
from tqdm import tqdm
import librosa

In [10]:
expression = ['afraid', 'angry', 'disgusted', 'happy', 'neutral', 'sad', 'surprised']

# データ前処理

## 画像

In [11]:
img_data_dir = '/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/KDEF/*'
img_data_names = glob(os.path.join(img_data_dir, '*'))
img_size = 32

In [12]:
#画像を感情ごとで分類、データセット作成
img_dataset = []
def create_img_dataset():
  for index, img_name in enumerate(img_data_names):  
    try:
      basename = os.path.basename(img_name)
      if basename[4:6] == 'AF':
        img_label = 0
      elif basename[4:6] == 'AN':
        img_label = 1
      elif basename[4:6] == 'DI':
        img_label = 2
      elif basename[4:6] == 'HA':
        img_label = 3
      elif basename[4:6] == 'NE':
        img_label = 4
      elif basename[4:6] == 'SA':
        img_label = 5
      else:
        img_label = 6

      #img_array = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)  # グレースケールで画像読み込み
      #img_resize_array = cv2.resize(img_array, (img_size, img_size))  # 画像のリサイズ
      img_dataset.append([img_name, img_label])  # 画像データ、ラベル情報を追加
    except Exception as e:
      pass



In [13]:
create_img_dataset()

In [14]:
multi_dataset = pd.DataFrame(index=[], columns=['img', 'voice', 'label'])
multi_dataset

,img,voice,label


In [15]:
for i, (img_name, label) in enumerate(img_dataset):
  multi_dataset.loc[i, ('img','label')] = [img_name, label]

multi_dataset

,img,voice,label
0,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,0
1,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,2
2,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,1
3,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,0
4,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,2
...,...,...,...
4905,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,6
4906,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,5
4907,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,5
4908,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,5


In [16]:
multi_dataset = multi_dataset.sort_values('label')
multi_dataset = multi_dataset.reset_index(drop=True)
multi_dataset

,img,voice,label
0,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,0
1,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,0
2,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,0
3,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,0
4,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,0
...,...,...,...
4905,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,6
4906,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,6
4907,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,6
4908,/content/drive/MyDrive/AI-app/KDEF_and_AKDEF/K...,NaN,6


## 音声

In [17]:
voice_data_dir = '/content/drive/MyDrive/AI-app/voice_data'
voice_data_names = []

In [ ]:
files = [
  os.path.join(voice_data_dir, 'Actor_*/03-01-06-*.wav'),#fear(afraid) 192
  os.path.join(voice_data_dir, 'Actor_*/03-01-05-*.wav'),#angry 192
  os.path.join(voice_data_dir, 'Actor_*/03-01-07-*.wav'),#disgust
  os.path.join(voice_data_dir, 'Actor_*/03-01-03-*.wav'),#happy
  os.path.join(voice_data_dir, 'Actor_*/03-01-01-*.wav'),#neutral 96
  os.path.join(voice_data_dir, 'Actor_*/03-01-04-*.wav'),#sad
  os.path.join(voice_data_dir, 'Actor_*/03-01-08-*.wav')#surprised
]

voice_data = np.hstack([
    np.array([
        librosa.feature.mfcc(*librosa.load(wav)).T for wav in glob(f)
    ]) for f in files
])

voice_label = np.hstack([
    np.full(len(glob(f)), i) for i, f in enumerate(files)
])

In [ ]:
from keras_preprocessing.sequence import pad_sequences
voice_data = pad_sequences(voice_data, 300, dtype=voice_data[0].dtype)

In [ ]:
wav = []
for f in files:
  wav.append(glob(f))

In [ ]:
wavs = []
for j in range(len(wav)):
  for i in range(len(wav[j])):
    wavs.append(wav[j][i])

wavs[:5]

In [ ]:
voices = []

for label in range(len(expression)):
  plus = (voice_label==label-1).sum() + (voice_label==label-2).sum() + (voice_label==label-3).sum() + (voice_label==label-4).sum() + (voice_label==label-5).sum() + (voice_label==label-6).sum()
  for index in range((multi_dataset['label']==label).sum()):
    voices.append(wavs[plus + (index%(voice_label==label).sum())])

In [ ]:
multi_dataset['voice'] = voices
multi_dataset

## データセットの処理

In [ ]:
multi_data_img = []
for img_name in multi_dataset['img'].to_numpy():
  img_array = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)  # グレースケールで画像読み込み
  img_resize_array = cv2.resize(img_array, (img_size, img_size))  # 画像のリサイズ
  multi_data_img.append(img_resize_array)
  print(img_name)
multi_data_img[0]

In [ ]:
multi_data_img_255 = np.array(multi_data_img).astype('float32') / 255
multi_data_img_255.shape

In [ ]:
multi_data_voice = []
multi_data_voice =np.hstack([
    np.array([
        librosa.feature.mfcc(*librosa.load(voice_name)).T for voice_name in multi_dataset['voice'].to_numpy()
    ])
])

In [ ]:
from keras_preprocessing.sequence import pad_sequences
multi_data_voice = pad_sequences(multi_data_voice, 300, dtype=multi_data_voice[0].dtype)

In [ ]:
multi_label = multi_dataset['label'].to_numpy().astype('int8')

# コード

## 画像

In [ ]:
from sklearn.model_selection import train_test_split

X_multi_img_train, X_multi_img_test, y_multi_img_train, y_multi_img_test = train_test_split(multi_data_img_255, multi_label, test_size=0.3, random_state=100, stratify=multi_label)

In [ ]:
from tensorflow.keras import utils as np_utils
y_test_org = y_multi_img_test
y_multi_img_train = np_utils.to_categorical(y_multi_img_train, len(expression))
y_multi_img_test = np_utils.to_categorical(y_multi_img_test, len(expression))

In [ ]:
multi_img_model = models.load_model('/content/drive/MyDrive/AI-app/models/img_an1_cnn_model.h5')

In [ ]:
# モデルを評価
score = multi_img_model.evaluate(X_multi_img_test, y_multi_img_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])

47/47 [==============================] - 3s 41ms/step - loss: 1.0162 - acc: 0.6246
正解率= 0.624575674533844 loss= 1.0162019729614258


## 音声

In [ ]:
from sklearn.model_selection import train_test_split

X_multi_voice_train, X_multi_voice_test, y_multi_voice_train, y_multi_voice_test = train_test_split(voice_data, voice_label, test_size=0.3, random_state=100, stratify=voice_label)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tensorflow.keras import layers

model = Sequential([
    LSTM(256, input_shape=(300, 20)),
    layers.Dropout(0.25),

    Dense(len(expression), activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [ ]:
X_multi_voice_train.dtype

dtype('O')

In [ ]:
y_multi_voice_train = y_multi_voice_train.astype('float32')
y_multi_voice_test = y_multi_voice_test.astype('float32')

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=5)

history = model.fit(X_multi_voice_train, y_multi_voice_train, epochs=100, validation_data=(X_multi_voice_test, y_multi_voice_test), callbacks=[early_stopping])

ValueError: ignored

## マルチモーダル1

## マルチモーダル2

## マルチモーダル3

# その他